In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append(r"C:\Users\wb557957\Desktop\GOSTnets-traffic") # this is the path to the new gostnets with load_traffic from Tom.
import GOSTnets as gn

import osmium, logging
import shapely.wkb as wkblib

from osgeo import ogr
from rtree import index
from shapely import speedups
from shapely.geometry import LineString, MultiLineString, MultiPoint, Point
from geopy.distance import vincenty
from boltons.iterutils import pairwise
from shapely.wkt import loads,dumps

In [2]:
pth = r"C:\Users\wb557957\Desktop\LKA"
inPth = r"C:\Users\wb557957\Desktop\LKA\lka_data"
outPth = r"C:\Users\wb557957\Desktop\LKA\lka_outputs"

kandy_osm = r'C:\Users\wb557957\Desktop\LKA\lka_data\kandy_roads.osm.pbf'

In [3]:
import GOSTnets.load_traffic as loadtrf

kandy = loadtrf.OSM_to_network(kandy_osm)

kandy

# error message: error adding edge between nodes between 35436481 and 4089554835
# between kandy and primrose. what happened here

finished with Osmium data extraction
35900
0


finished building node edge lists
all_edges length
1114426
finished building node and edge GeoDataFrames
all_edges_gdf length
1114426


In [5]:
# load csv of three combined mapbox edges
kandy_mb = r'C:\Users\wb557957\Desktop\LKA\lka_MB_osmAttributes.csv'

In [6]:
kandy.roads_raw.infra_type.value_counts()

residential       376288
unclassified      220690
tertiary          143409
track             140963
secondary          81347
service            46869
trunk              38055
primary            21128
path               18391
living_street      17733
footway             5375
road                2645
construction         697
steps                371
trunk_link           166
tertiary_link        102
secondary_link        44
primary_link          38
pedestrian            36
raceway               30
motorway_link         19
bridleway             13
cycleway              11
services               6
Name: infra_type, dtype: int64

In [7]:
kandy.nodes_raw

,osm_id,geometry
0,35436481,POINT (80.63332 7.29317)
1,2078304672,POINT (80.63317 7.29309)
2,3810950074,POINT (80.63309 7.29306)
3,5504401132,POINT (80.63306 7.29305)
4,3057458327,POINT (80.63287 7.29295)
...,...,...
1150414,7438421225,POINT (80.51874 7.04207)
1150415,7438421292,POINT (80.51871 7.04210)
1150416,7438421291,POINT (80.51868 7.04211)
1150417,7438421290,POINT (80.51866 7.04211)


In [8]:
accepted_road_types = ['residential', 
                       'unclassified', 
                       'track',
                       'service',
                       'tertiary',
                       'road',
                       'secondary',
                       'primary',
                       'trunk',
                       'primary_link',
                       'trunk_link',
                       'tertiary_link',
                       'secondary_link']

In [9]:
kandy.filterRoads(acceptedRoads = accepted_road_types)

In [10]:
kandy.roads_raw.infra_type.value_counts()

residential       376288
unclassified      220690
tertiary          143409
track             140963
secondary          81347
service            46869
trunk              38149
primary            21128
road                2645
trunk_link           166
tertiary_link        102
secondary_link        44
primary_link          38
Name: infra_type, dtype: int64

In [12]:
kandy.roads_raw

,stnode,endnode,osm_id,infra_type,geometry
0,35436481,2078304672,5131121,trunk,"LINESTRING (80.63332 7.29317, 80.63317 7.29309)"
1,2078304672,3810950074,5131121,trunk,"LINESTRING (80.63317 7.29309, 80.63309 7.29306)"
2,3810950074,5504401132,5131121,trunk,"LINESTRING (80.63309 7.29306, 80.63306 7.29305)"
3,5504401132,3057458327,5131121,trunk,"LINESTRING (80.63306 7.29305, 80.63287 7.29295)"
4,3057458327,2266684575,5131121,trunk,"LINESTRING (80.63287 7.29295, 80.63260 7.29281)"
...,...,...,...,...,...
1114515,7438421225,7438421292,795283912,residential,"LINESTRING (80.51874 7.04207, 80.51871 7.04210)"
1114516,7438421292,7438421291,795283912,residential,"LINESTRING (80.51871 7.04210, 80.51868 7.04211)"
1114517,7438421291,7438421290,795283912,residential,"LINESTRING (80.51868 7.04211, 80.51866 7.04211)"
1114518,7438421290,7438421289,795283912,residential,"LINESTRING (80.51866 7.04211, 80.51860 7.04210)"


In [13]:
kandy_mb_edges = r'C:\Users\wb557957\Desktop\LKA\lka_MB_osmAttributes.csv'

In [18]:
kandy.apply_traffic_speeds_to_roads_raw(r"C:\Users\wb557957\Desktop\LKA\lka_data\1233302-Asia-Colombo.csv")

1
C:\Users\wb557957\Desktop\LKA\lka_data\1233302-Asia-Colombo.csv
finished reading C:\Users\wb557957\Desktop\LKA\lka_data\1233302-Asia-Colombo.csv into dataframe
calculating min, max, and mean values.
finished calculating min, max, and mean values. Printing traffic_simplified head
    FROM_NODE     TO_NODE  min_speed  max_speed  mean_speed
0   102738212  3279663552       34.0       57.0   45.918651
1  1093698450  1093697790       34.0       34.0   34.000000
2  1093698671  1093697934       34.0       34.0   34.000000
3  1093798472  1093799200       39.0       39.0   39.000000
4  1093798791  1093799519       31.0       34.0   33.910714
adding the traffic speeds to the edges


KeyError: ('mean_speed', 'occurred at index 0')

In [ ]:
kandy.generateRoadsGDF()